In [36]:
import pandas as pd

games = pd.read_csv('./data/games_1500.csv')
print(games)

      winner  white_rating  black_rating  \
0      white          1523          1469   
1      white          1520          1423   
2      black          1413          2108   
3      black          1381          1867   
4      black          1381          1936   
...      ...           ...           ...   
12234  white          1550          1250   
12235  black          1263          1770   
12236  white          1759          1265   
12237  white          1523          1256   
12238  white          1691          1220   

                                                   moves  
0      e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...  
1      d4 d5 e4 dxe4 Nc3 Nf6 f3 exf3 Nxf3 Nc6 Bb5 a6 ...  
2                         e4 Nc6 d4 e5 d5 Nce7 c3 Ng6 b4  
3      e4 e6 d4 d5 e5 c5 c3 Nc6 Nf3 Qb6 Be3 Qxb2 Nbd2...  
4      e4 e6 Nf3 d5 exd5 exd5 Qe2+ Be7 Nc3 Nf6 d4 O-O...  
...                                                  ...  
12234  e4 e5 Nf3 Nc6 Bc4 Bc5 b4 Bxb4 c3 Ba5 Qb3 Qf6 O...  

In [37]:
# games_transformed = pd.DataFrame(columns=['T1', 'T10'])
# games_transformed.loc[0] = [['a','b','c'],1]
# games_transformed.loc[1] = [['a','b','c','d'],6]
# print(games_transformed)
# games_transformed.to_csv("test.csv")

# columns titles
positions = ['p'+str(i) for i in range(64)]
columns = positions
columns += ['player']
columns += ['move']
print(columns)

['p0', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11', 'p12', 'p13', 'p14', 'p15', 'p16', 'p17', 'p18', 'p19', 'p20', 'p21', 'p22', 'p23', 'p24', 'p25', 'p26', 'p27', 'p28', 'p29', 'p30', 'p31', 'p32', 'p33', 'p34', 'p35', 'p36', 'p37', 'p38', 'p39', 'p40', 'p41', 'p42', 'p43', 'p44', 'p45', 'p46', 'p47', 'p48', 'p49', 'p50', 'p51', 'p52', 'p53', 'p54', 'p55', 'p56', 'p57', 'p58', 'p59', 'p60', 'p61', 'p62', 'p63', 'player', 'move']


In [93]:
from chess_transformation import *

games_transformed = pd.DataFrame(columns=columns)

index = 0
for i in range(len(games)):
# for i in range(1):
    print(str(i/len(games)*100)+'%', end='\r')
    boards = []
    move_list = []
    game = games.iloc[0]

    moves = game['moves'].split(' ')
    winner = game['winner']
    # print(game['moves'])
    # print(moves)
    # print(winner)

    for i in range(len(moves)-1):
        if winner == 'white' and (i%2==0 or i == 0):
            boards.append(algebraic_to_matrix_python_chess(moves[:i]))
            move_list.append(moves[i])
        elif winner == 'black' and i%2==1 and i != 0:
            boards.append(algebraic_to_matrix_python_chess(moves[:i]))
            move_list.append(moves[i])

    for i in range(len(boards)):
        # print(board_to_linear(boards[i])+[winner]+[move_list[i]])
        # print('----------------------------------------')
        # print_board(boards[i])
        # print('next:', move_list[i])
        games_transformed.loc[index] = board_to_linear(boards[i])+[winner]+[move_list[i]]
        index+=1

        if index%100 == 0: # save every 100 games
            games_transformed.to_csv("./data/games_cleaned_1500.csv")

# games_transformed.to_csv("test.csv")
games_transformed.to_csv("./data/games_cleaned_1500.csv")
print(games_transformed)

KeyboardInterrupt: 